
# A widget to reproduce the figures in Hartmut Zohm's paper, "On the size of tokamak fusion power plants"
published in Philosophical Transactions A, 04 February 2019, 
http://dx.doi.org/10.1098/rsta.2017.0437

This widget allows a user to interactively explore the model described by H. Zohm in the above paper. 

Instructions: Execute all the cells, then find the interactive widget at the bottom.

In [13]:
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from copy import copy

In [14]:
class ReactorScaling:
    """
    This class just serves as a convenient collection of variables.
    (Could probably be replaced by a dict.)
    """
    def __init__(self, A, q, H, beta_N, f_GW, f_LH, P_fus, Q_CD, Q_PB):
        self.A = A               # Aspect ratio
        self.q = q               # safety factor
        self.H = H               # H-factor
        self.beta_N = beta_N     # normalized β
        self.f_GW = f_GW         # Greenwald fraction
        self.f_LH = f_LH         # particle losses compared with the L-H transition point 
        self.P_fus = P_fus       # fusion power
        self.Q_CD = Q_CD         # P_fus / current drive power
        self.Q_PB = Q_PB         # P_fus / auxilliary power
        
    def set_B_and_R_ref(self, B_ref, R_ref):
        self.B = B_ref           # ITER's on-axis B field
        self.R = R_ref           # ITER plasma radius R_0


Here are ITER's specifications as reported in the paper.

In [15]:
RS_ITER = ReactorScaling(A=3.1, q=3.1, H=1.0, beta_N=1.8, f_GW=0.85, f_LH=1.33, P_fus=400, Q_CD=1, Q_PB=10)
RS_ITER.set_B_and_R_ref(5.2, 6.2)
# note: I call this RS_ITER because 'iter' is a python builtin function

$P_\mathrm{fus}$ is the fusion power produced in the reactor.

$$P_\mathrm{fus} = c_\mathrm{fus} \frac{\beta_N^2 B^4 A^3}{q^2 A^4} \qquad \textrm{(Eq. 2.1)}$$
Use ITER's specifications we can solve for $c_\mathrm{fus}$, then, 
solve for $B$ in terms of the other variables. 
$$B = \frac{P_\mathrm{fus}^{1/4} q^{1/2} A}{c_\mathrm{fus}^{1/4} \beta_N^{1/2} R^{3/4}} $$


In [16]:
def B_Pfus(R, reactor, reactor_ref):
    """ Equation 2.1, solved for B.
    
        R: major radius (or a numpy array of radii)
        reactor: physics and engineering parameters for the reactor to be explored
        reactor_ref: physics and engineering parameters for a reference reactor 

        Returns: a value of B_0 (or an array of B_0) required for the reactor
    """
    P_fus_ref = reactor_ref.P_fus
    A_ref     = reactor_ref.A
    q_ref     = reactor_ref.q
    beta_N_ref= reactor_ref.beta_N
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    P_fus     = reactor.P_fus
    q         = reactor.q
    A         = reactor.A
    beta_N    = reactor.beta_N
    
    c_fus = P_fus_ref * q_ref**2 * A_ref**4 / (beta_N_ref**2 * B_ref**4 * R_ref**3)
    numerator = P_fus**0.25 * q**0.5 * A
    denominator = c_fus ** 0.25 * beta_N**0.5 * R**(0.75)
    return numerator/denominator

$Q_{\mathrm{PB}}$ is the ratio of the fusion power to the auxiliary power needed to sustain the plasma,

$$ Q_{\mathrm{PB}} = \frac{P_\mathrm{fus}}{P_\mathrm{AUX}} = \frac{1}{c_\mathrm{PB}\left(q^{3.1} A^{3.53} H^{-3.23} \beta_N^{-0.1} R^{-2.7} B^{-3.7} - 1/5\right)} \qquad \textrm{(Eq. 2.1)}$$
The $1/5$ there is because $1/5$ of the fusion power is in the form of the $\alpha$ particles which heat the plasma.
Using the typical ITER value of $Q_\mathrm{PB} = 10$ we can solve for $c_\mathrm{PB}$, then solve for $B$ as a function of a _new_ set of reactor parameters.

$$B = \left(\frac{c_\mathrm{PB}}{\left(Q_\mathrm{PB}^{-1} + 1/5\right)} \frac{q^{3.1} A^{3.53}}{H^{3.23}\beta_N^{0.1}R^{2.7}}\right)^{1/3.7}$$

In [17]:
def B_Q_PB(R, reactor, reactor_ref):
    """ Equation 2.2, solved for B
    
        R: major radius (or a numpy array of radii)
        reactor: physics and engineering parameters for the reactor to be explored
        reactor_ref: physics and engineering parameters for a reference reactor 

        Returns: a value of B_0 (or an array of B_0) required for the reactor
    """
    Q_PB_ref  = reactor_ref.Q_PB
    H_ref     = reactor_ref.H
    A_ref     = reactor_ref.A
    q_ref     = reactor_ref.q
    beta_N_ref= reactor_ref.beta_N
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    Q_PB      = reactor.Q_PB
    q         = reactor.q
    A         = reactor.A
    H         = reactor.H
    beta_N    = reactor.beta_N
    
    c_PB = (Q_PB_ref**(-1) + 1/5) / (q_ref**3.1 * A_ref**3.53 / \
                                     (H_ref**3.23 * beta_N_ref**0.1 * R_ref**2.7 * B_ref**3.7))
    
    first_term = (c_PB / (Q_PB**(-1) + 1/5))
    numerator = (q**(3.1) * A**(3.53))
    denominator = (H**(3.23) * beta_N**(0.1) * R**(2.7))
    return (first_term * numerator / denominator)**(1/3.7)

$Q_\mathrm{CD}$ is the ratio of fusion power to the current drive power needed to sustain the plasma current. The paper assumes that the tokamak operates in steady state: if the requirement is not met (as for ITER itself), that means the tokamak can only operate in a pulsed mode.

$$ Q_\mathrm{CD} = \frac{P_\mathrm{fus}}{P_\mathrm{CD}} = c_\mathrm{CD} \frac{\beta_N^{3} R^3 B^3}{A^3 f_\mathrm{GW}^2\left( 5 + Z_\mathrm{eff}\right)\left(1 - c_\mathrm{BS} A^{1/2} q \beta_N\right)}$$

The value of $c_\mathrm{BS}$ is found by comparing formulas with Zohm's [2017 paper](https://doi.org/10.1088/1741-4326/aa739e). See that paper's appendix: it equals $c_9 c_4 / c_3 = 0.02228$.

The paper assumes that $Z_\mathrm{eff} \ll 5$ and does not give a value for it, so we assume that it is unchanged from the ITER value. We can solve for B as a function of the other variables, scaled from the reference (ITER) value:

$$ B = B_\mathrm{ref} \left(\frac{Q_\mathrm{CD}}{Q_\mathrm{CD, ref}}\right)^{1/3}
\left(\frac{A}{A_\mathrm{ref}}\right)
\left(\frac{R_\mathrm{ref}}{R}\right)
\left(\frac{\beta_{N,\mathrm{ref}}}{\beta_N}\right)
\left(\frac{f_\mathrm{GW}}{f_{\mathrm{GW,ref}}}\right)^{2/3}
\left(\frac{1 - c_\mathrm{BS} A^{1/2} q \beta_N}{1 - c_\mathrm{BS} A^{1/2}_\mathrm{ref} q_\mathrm{ref} \beta_{N,\mathrm{ref}}}\right)^{1/3}
$$

In [18]:
def B_Q_CD(R, reactor, reactor_ref):
    """ Equation 2.3, solved for B
    
        R: major radius (or a numpy array of radii)
        reactor: physics and engineering parameters for the reactor to be explored
        reactor_ref: physics and engineering parameters for a reference reactor 

        Returns: a value of B_0 (or an array of B_0) required for the reactor
    """
    Q_CD_ref  = reactor_ref.Q_CD
    f_GW_ref  = reactor_ref.f_GW
    A_ref     = reactor_ref.A
    q_ref     = reactor_ref.q
    beta_N_ref= reactor_ref.beta_N
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    Q_CD      = reactor.Q_CD
    f_GW      = reactor.f_GW
    q         = reactor.q
    A         = reactor.A
    beta_N    = reactor.beta_N
    
    Z_eff_ref = 1 # These are considered to be << 5, and unchanged from the ITER values.
    Z_eff = Z_eff_ref 
    c_BS = 0.7 * 0.00331/0.104
    return B_ref * (Q_CD / Q_CD_ref)**(1/3) * (A/A_ref) * (R_ref / R) * \
        (beta_N_ref / beta_N) * (f_GW / f_GW_ref) **(2/3) * ((5 + Z_eff)/(5 + Z_eff_ref))**(1/3) * \
        ((1 - c_BS * A**0.5 * q * beta_N) / (1 - c_BS * A_ref**0.5 * q_ref * beta_N_ref))**(1/3)

Zohm notes that "For a fully-consistent solution, $Q_{CD} = Q_{PB}$ but as the two quantities assume large values, their difference will not influence the power balance strongly."

$f_{z, \mathrm{div}}$ represents the concentration of impurities in the divertor. It's not explicitly calculated in Zohm's paper; instead, the equation is scaled relative to the (non-calculated) ITER value.

$$f_{Z, \mathrm{div}} = c_\mathrm{exh} \frac{f_{\mathrm{LH}}^{1.14}q^{0.32} B^{0.88} R^{1.33}}{f_\mathrm{GW}^{1.18}} \qquad \textrm{Eq. 2.4}$$

The scaling equation relative to the (ITER) reference point is

$$ B = B_\mathrm{ref} \left(\left(\frac{f_\mathrm{GW}}{f_{\mathrm{GW},\mathrm{ref}}}\right)^{1.18}
\left(\frac{f_{\mathrm{LH},\mathrm{ref}}}{f_\mathrm{LH}}\right)^{1.14}
\left(\frac{q_\mathrm{ref}}{q}\right)^{0.32}
\left(\frac{R_\mathrm{ref}}{R}\right)^{1.33}\right)^{1/0.88}
$$

In [20]:
def B_f_zdiv(R, reactor, reactor_ref):
    """ Equation 2.4 solved for B
    
        R: major radius (or a numpy array of radii)
        reactor: physics and engineering parameters for the reactor to be explored
        reactor_ref: physics and engineering parameters for a reference reactor 

        Returns: a value of B_0 (or an array of B_0) required for the reactor
    """
    f_GW_ref  = reactor_ref.f_GW
    f_LH_ref  = reactor_ref.f_LH
    q_ref     = reactor_ref.q
    B_ref     = reactor_ref.B
    R_ref     = reactor_ref.R
    
    f_GW      = reactor.f_GW
    f_LH      = reactor.f_LH
    q         = reactor.q
    
    B = B_ref * ((f_GW / f_GW_ref)**(1.18) * (f_LH_ref/f_LH)**(1.14) * \
         (q_ref / q)**(0.32) * (R_ref / R)**(1.33))**(1/0.88)
    return B

## Make an interactive plot

Plot as a function of $R$ the required $B$ to achieve the set engineering values of $P_\mathrm{fus}, Q_\mathrm{PB}$, and $Q_\mathrm{CD}$, as well as a $f_Z$ value equivalent to that in the reference reactor, depending on the specified physics parameters $f_\mathrm{LH}$, $f_\mathrm{GW}$, $q$, $H$, and $\beta_N$.

In [21]:
R = np.arange(2, 10.1, 0.25)
plt.rcParams['font.size'] = 18

def f(my_P_fus, my_Q_PB, my_Q_CD, my_f_LH, my_f_GW, my_q, my_H, my_beta_N):
    reactor= ReactorScaling(3.1, my_q, my_H, my_beta_N, my_f_GW, my_f_LH, my_P_fus, my_Q_CD, my_Q_PB)
    bs_pfus = B_Pfus(R, reactor, RS_ITER)
    bs_QPB  = B_Q_PB(R, reactor, RS_ITER)
    bs_qcd = B_Q_CD(R, reactor, RS_ITER)
    bs_fz = B_f_zdiv(R, reactor, RS_ITER)
    
    
    fig, ax = plt.subplots()
    ax.plot(R, bs_pfus, label=r'$P_\mathrm{fus}$', color='red')
    ax.plot(R, bs_QPB, label=r'$Q_\mathrm{PB}$', color='blue')
    ax.plot(R, bs_qcd, label=r'$Q_\mathrm{CD}$', color='green')
    ax.plot(R, bs_fz, label=r'$f_Z$', color='yellow')

    plt.ylim(5, 14)
    plt.xlim(2,10)
    plt.xlabel(r'$R/\mathrm{m}$')
    plt.ylabel(r'$B/\mathrm{T}$')
    plt.legend()

    plt.show()

interactive_plot = interactive(f, my_f_LH=widgets.FloatSlider(description=r'$f_\mathrm{LH}$', min=1.0, max=1.33, step=0.01, value=1.33),
                               my_f_GW=widgets.FloatSlider(description=r'$f_\mathrm{GW}$', min=0.85, max=1.2, step=0.05, value=0.85),
                               my_q=widgets.FloatSlider(description=r'$q$', min=3.1, max=6, step=0.1, value=3.1),
                               my_H=widgets.FloatSlider(description=r'$H$', min=1.0, max=1.2, step=0.1, value=1.0),
                               my_P_fus=widgets.IntSlider(description=r'$P_\mathrm{fus}$', min=400, max=3500, step=100, value=400),
                               my_Q_PB=widgets.IntSlider(description=r'$Q_\mathrm{PB}$', min=1, max=100, step=1, value=10), 
                               my_Q_CD=widgets.IntSlider(description=r'$Q_\mathrm{CD}$', min=1, max=100, step=1, value=10),
                               my_beta_N=widgets.FloatSlider(description=r'$\beta_\mathrm{N}$', min=1.8, max=3.5, step=0.1, value=1.8))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=400, description='$P_\\mathrm{fus}$', max=3500, min=400, step=100), IntS…

The window of feasible reactors is the space _above_ the $P_\mathrm{fus}$, $Q_\mathrm{PB}$, and $Q_\mathrm{CD}$ curves, and _below_ the $f_Z$ curve. Not all settings result in a possible reactor.

The default settings for the plot reproduce Figure 1.
Try changing the sliders to the values for the other figures!